In [1]:
import tensorflow as tf
import numpy as np
import glob
import json
from collections import Counter
from tqdm import tqdm
from snownlp import SnowNLP
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

a = tf.test.is_built_with_cuda()
b = tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)     
print(a)
print(b)
np.random.seed(1000)

True
True


In [2]:
def split_dataset():
    paths = glob.glob('./dataset/json/poet.*.json')
    poets = []
    for path in paths:
        data = open(path, 'r', encoding='utf-8').read()
        data = json.loads(data)
        #print(data)
        for item in data:
            content = ''.join(item['paragraphs'])
            if len(content)>= 24 and len(content) <= 32:
                content = SnowNLP(content)
                content=content.han
                if len(content)%4==0:
                    poets.append('[' + content + ']')
    print('We have %d Chinese ancient poets' % len(poets), poets[0], poets[-1])
    training_data, validation_data = split(poets, test_size = 0.20, random_state = 1000)

    return training_data, validation_data


In [3]:
training_data, validation_data = split_dataset()

We have 104147 Chinese ancient poets [欲出未出光辣达，千山万山如火发。须臾走向天上来，逐却残星赶却月。] [书劒催人不暂闲，洛阳羁旅复秦关。容颜岁岁愁边改，乡国时时梦里还。]


In [4]:
print(len(training_data))

83317


In [2]:
def prepocess_data(dataset):
    chars = []
    for item in dataset:
        chars += [c for c in item]
    print('We have %d words' % len(chars))

    chars = sorted(Counter(chars).items(), key=lambda x:x[1], reverse=True)
    print('We have %d different words in dataset' % len(chars))
    chars = [c[0] for c in chars]
    char2id = {c: i + 1 for i, c in enumerate(chars)}
    id2char = {i + 1: c for i, c in enumerate(chars)}
    
    return char2id, id2char


In [6]:
def process_data(char2id, id2char, dataset, ga_individual_solution):   
# Decode GA solution to integer for batch_size and hidden_size
#     batch_size_bits = BitArray(ga_individual_solution[0:8])
#     batch_size = batch_size_bits.uint
#     print('\nBatch Size: ', batch_size)
    batch_size = 512
    X_data = []
    Y_data = []

    for b in range(len(dataset) // batch_size):
        start = b * batch_size
        end = b * batch_size + batch_size
        batch = [[char2id[c] for c in dataset[i]] for i in range(start, end)]
        maxlen = max(map(len, batch))
        X_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        Y_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        for i in range(batch_size):
            X_batch[i, :len(batch[i]) - 1] = batch[i][:-1]
            Y_batch[i, :len(batch[i]) - 1] = batch[i][1:]
        X_data.append(X_batch)
        Y_data.append(Y_batch)
    return X_data, Y_data
    
    

In [10]:
def build_model_and_train_evaluate(ga_individual_solution):
    #training_data, validation_data = split_dataset()
    training_char2id, training_id2char = prepocess_data(training_data)
    validation_char2id, validation_id2char = prepocess_data(validation_data)
    
    import pickle
    with open('dictionary.pkl', 'wb') as fw:
        pickle.dump([training_char2id, training_id2char], fw)
    
    X_training_data, Y_training_data = process_data(training_char2id, training_id2char, training_data, ga_individual_solution)
    X_validation_data, Y_validation_data = process_data(validation_char2id, validation_id2char, validation_data, ga_individual_solution)
    
    tf.reset_default_graph()
    batch_size = 512
    num_layer = 2
    embedding_size = 256
    
    hidden_size = []
    hidden_size_bits_1 = BitArray(ga_individual_solution[0:8])
    hidden_size.append(hidden_size_bits_1.uint)
    print('\nHidden Size 1: ', hidden_size[0])
    hidden_size_bits_2 = BitArray(ga_individual_solution[8:])
    hidden_size.append(hidden_size_bits_2.uint)
    print('\nHidden Size 2: ', hidden_size[1])
    
    
    X_training= tf.placeholder(tf.int32, [batch_size, None])
    Y_training= tf.placeholder(tf.int32, [batch_size, None])
    learning_rate = tf.Variable(0.0, trainable=False)
    ikeep_prob = tf.placeholder(tf.float32, name='ikeep_prob')
    okeep_prob = tf.placeholder(tf.float32, name='okeep_prob')
    cell = tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_size[i]), input_keep_prob=ikeep_prob, output_keep_prob=ikeep_prob )for i in range(num_layer)], 
        state_is_tuple=True)

    initial_state = cell.zero_state(batch_size, tf.float32)
    tr_embeddings = tf.Variable(tf.random_uniform([len(training_char2id) + 1, embedding_size], -1.0, 1.0))
    tr_embedded = tf.nn.embedding_lookup(tr_embeddings, X_training)
    tr_outputs, tr_last_states = tf.nn.dynamic_rnn(cell, tr_embedded, initial_state=initial_state)
    tr_outputs = tf.reshape(tr_outputs, [-1, hidden_size[1]])                # batch_size * max_time, hidden_size

    training_logits = tf.layers.dense(tr_outputs, units=len(training_char2id) + 1)       # batch_size * max_time, len(char2id) + 1
    training_logits = tf.reshape(training_logits, [batch_size, -1, len(training_char2id) + 1]) # batch_size, max_time, len(char2id) + 1
    probs = tf.nn.softmax(training_logits)                                   # batch_size, max_time, len(char2id) + 1
    loss = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(training_logits, Y_training, tf.ones_like(Y_training, dtype=tf.float32)))
    params = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
    optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    Loss=[]
    Validation_loss=[]
    for epoch in range(40):
        sess.run(tf.assign(learning_rate, 0.0004 * (0.97 ** epoch)))

        data_index = np.arange(len(X_training_data))
        np.random.shuffle(data_index)
        X_training_data = [X_training_data[i] for i in data_index]
        Y_training_data = [Y_training_data[i] for i in data_index]

        losses = []
        validates=[]

        for i in tqdm(range(len(X_training_data))):
            ls_,  _ = sess.run([loss, optimizer], feed_dict={X_training: X_training_data[i], Y_training: Y_training_data[i], ikeep_prob: 0.8, okeep_prob: 0.8})
            losses.append(ls_)
        for i in tqdm(range(len(X_validation_data))):
            validate_acc = sess.run([loss], feed_dict={X_training: X_validation_data[i], Y_training: Y_validation_data[i], ikeep_prob: 1, okeep_prob: 1})
            validates.append(validate_acc)
            
        
        Loss.append(np.mean(losses))
        Validation_loss.append(np.mean(validates))
    #save model
    saver = tf.train.Saver()
    saver.save(sess, './50/50_epoch')
        
    #Now the model has been trained and we want it to generate a poem
    with open('dictionary.pkl', 'rb') as fr:
        [char2id, id2char] = pickle.load(fr)
    
    tf.reset_default_graph()
    test_batch_size = 1

    X = tf.placeholder(tf.int32, [test_batch_size, None])
    Y = tf.placeholder(tf.int32, [test_batch_size, None])
    learning_rate = tf.Variable(0.0, trainable=False)

    cell = tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.LSTMCell(hidden_size[i], state_is_tuple=True) for i in range(num_layer)], 
        state_is_tuple=True)
    initial_state = cell.zero_state(test_batch_size, tf.float32)

    embeddings = tf.Variable(tf.random_uniform([len(char2id) + 1, embedding_size], -1.0, 1.0))
    embedded = tf.nn.embedding_lookup(embeddings, X)

    outputs, last_states = tf.nn.dynamic_rnn(cell, embedded, initial_state=initial_state)

    outputs = tf.reshape(outputs, [-1, hidden_size[1]])
    logits = tf.layers.dense(outputs, units=len(char2id) + 1)
    probs = tf.nn.softmax(logits)
    targets = tf.reshape(Y, [-1])

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))
    params = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
    optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./50'))
    
    def poetry_generate_with_topic(head):
        states_ = sess.run(initial_state)
        gen = ''
        c = '['
        i = 0
        while c != ']':
            gen += c
            x = np.zeros((test_batch_size, 1))
            x[:, 0] = char2id[c]
            probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
            probs_ = np.squeeze(probs_)
            pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
            while pos not in id2char:
                pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
            
            c = id2char[pos]
            if i<len(head):
                c=head[i]
                i+=1
                
        return gen[1:]
    
    print('Validation_loss = ', Validation_loss[-1])
    print('Loss = ', Loss[-1])
    print(poetry_generate_with_topic('边塞'))
    return Loss[-1],


In [11]:
population_size = 8
num_generations = 8
gene_length = 16

# As we are trying to minimize the Loss and the Validation_loss, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMulti', base.Fitness, weights = (-1.0, ))
creator.create('Individual', list , fitness = creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', build_model_and_train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)

C:\Users\willy\AppData\Roaming\Python\Python36\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\willy\AppData\Roaming\Python\Python36\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  172

Hidden Size 2:  216


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  7.15it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.346566
Loss =  4.8884296
边塞堂坞讲春色，风浸清晴夜有迟。殿上一沾仙野客，春来秋色不忘舟。
We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  193

Hidden Size 2:  177


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  7.42it/s]


...

INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.2681184
Loss =  5.013804
边塞何须百中，戍棠遮半阴回。老僧迷复远起，不在眼边空月。
We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  112

Hidden Size 2:  184


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  7.78it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.2589593
Loss =  4.958887
边塞不在石龙楼，不为弓翼已万头。却射俗翁休博迅，却勤家老转增僊。
We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  234

Hidden Size 2:  237


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:06<00:00,  6.66it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.3651724
Loss =  4.8598657
边塞楼峰出斗中，翠天元道水青旂。黄荒犹肉伤船妇，却趁黄阙劒上船。


In [12]:
best_individuals = tools.selBest(population,k = 1)
best_hidden_size_1 = None
best_hidden_size_2 = None

for bi in best_individuals:
    best_hiden_size_1_bits = BitArray(bi[0:8])
    best_hidden_size_2_bits = BitArray(bi[8:]) 
    best_hidden_size_1 = best_hiden_size_1_bits.uint
    best_hidden_size_2 = best_hidden_size_2_bits.uint
    print('\nHidden Size 1 : ', best_hidden_size_1, ', Hidden Size 2: ', best_hidden_size_2)


Hidden Size 1 :  240 , Hidden Size 2:  237


In [3]:
def process_data(char2id, id2char, dataset):   
# Decode GA solution to integer for batch_size and hidden_size
#     batch_size_bits = BitArray(ga_individual_solution[0:8])
#     batch_size = batch_size_bits.uint
#     print('\nBatch Size: ', batch_size)
    batch_size = 512
    X_data = []
    Y_data = []

    for b in range(len(dataset) // batch_size):
        start = b * batch_size
        end = b * batch_size + batch_size
        batch = [[char2id[c] for c in dataset[i]] for i in range(start, end)]
        maxlen = max(map(len, batch))
        X_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        Y_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        for i in range(batch_size):
            X_batch[i, :len(batch[i]) - 1] = batch[i][:-1]
            Y_batch[i, :len(batch[i]) - 1] = batch[i][1:]
        X_data.append(X_batch)
        Y_data.append(Y_batch)
    return X_data, Y_data

In [4]:
#Now we need to train a neural network with these optimized parameters with more epochs and the whole dataset

In [5]:
def split_dataset():
    paths = glob.glob('./dataset/json/poet.*.json')
    poets = []
    for path in paths:
        data = open(path, 'r', encoding='utf-8').read()
        data = json.loads(data)
        #print(data)
        for item in data:
            content = ''.join(item['paragraphs'])
            if len(content)>= 24 and len(content) <= 32:
                content = SnowNLP(content)
                content=content.han
                if len(content)%4==0:
                    poets.append('[' + content + ']')
    print('We have %d Chinese ancient poets' % len(poets), poets[0], poets[-1])
    training_data, validation_data = split(poets, test_size = 0.000001, random_state = 1000)

    return training_data, validation_data

In [6]:
training_data, validation_data = split_dataset()

We have 104147 Chinese ancient poets [欲出未出光辣达，千山万山如火发。须臾走向天上来，逐却残星赶却月。] [书劒催人不暂闲，洛阳羁旅复秦关。容颜岁岁愁边改，乡国时时梦里还。]


In [8]:
training_char2id, training_id2char = prepocess_data(training_data)
validation_char2id, validation_id2char = prepocess_data(validation_data)

import pickle
with open('dictionary.pkl', 'wb') as fw:
    pickle.dump([training_char2id, training_id2char], fw)
X_training_data, Y_training_data = process_data(training_char2id, training_id2char, training_data)
X_validation_data, Y_validation_data = process_data(validation_char2id, validation_id2char, validation_data)

tf.reset_default_graph()
batch_size = 512
num_layer = 2
embedding_size = 256

hidden_size = [240, 237]

X_training= tf.placeholder(tf.int32, [batch_size, None])
Y_training= tf.placeholder(tf.int32, [batch_size, None])
learning_rate = tf.Variable(0.0, trainable=False)
ikeep_prob = tf.placeholder(tf.float32, name='ikeep_prob')
okeep_prob = tf.placeholder(tf.float32, name='okeep_prob')
cell = tf.nn.rnn_cell.MultiRNNCell(
    [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_size[i]), input_keep_prob=ikeep_prob, output_keep_prob=ikeep_prob )for i in range(num_layer)], 
    state_is_tuple=True)

initial_state = cell.zero_state(batch_size, tf.float32)
tr_embeddings = tf.Variable(tf.random_uniform([len(training_char2id) + 1, embedding_size], -1.0, 1.0))
tr_embedded = tf.nn.embedding_lookup(tr_embeddings, X_training)
tr_outputs, tr_last_states = tf.nn.dynamic_rnn(cell, tr_embedded, initial_state=initial_state)
tr_outputs = tf.reshape(tr_outputs, [-1, hidden_size[1]])                # batch_size * max_time, hidden_size

training_logits = tf.layers.dense(tr_outputs, units=len(training_char2id) + 1)       # batch_size * max_time, len(char2id) + 1
training_logits = tf.reshape(training_logits, [batch_size, -1, len(training_char2id) + 1]) # batch_size, max_time, len(char2id) + 1
probs = tf.nn.softmax(training_logits)                                   # batch_size, max_time, len(char2id) + 1
loss = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(training_logits, Y_training, tf.ones_like(Y_training, dtype=tf.float32)))
params = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
Loss=[]
Validation_loss=[]
for epoch in range(50):
    sess.run(tf.assign(learning_rate, 0.0004 * (0.97 ** epoch)))

    data_index = np.arange(len(X_training_data))
    np.random.shuffle(data_index)
    X_training_data = [X_training_data[i] for i in data_index]
    Y_training_data = [Y_training_data[i] for i in data_index]

    losses = []
    validates=[]

    for i in tqdm(range(len(X_training_data))):
        ls_,  _ = sess.run([loss, optimizer], feed_dict={X_training: X_training_data[i], Y_training: Y_training_data[i], ikeep_prob: 0.8, okeep_prob: 0.8})
        losses.append(ls_)
#     for i in tqdm(range(len(X_validation_data))):
#         validate_acc = sess.run([loss], feed_dict={X_training: X_validation_data[i], Y_training: Y_validation_data[i], ikeep_prob: 1, okeep_prob: 1})
#         validates.append(validate_acc)
    Loss.append(np.mean(losses))
    Validation_loss.append(np.mean(validates))
#save model
saver = tf.train.Saver()
saver.save(sess, './GA/50_epoch')

#Now the model has been trained and we want it to generate a poem
with open('dictionary.pkl', 'rb') as fr:
    [char2id, id2char] = pickle.load(fr)

tf.reset_default_graph()
test_batch_size = 1

X = tf.placeholder(tf.int32, [test_batch_size, None])
Y = tf.placeholder(tf.int32, [test_batch_size, None])
learning_rate = tf.Variable(0.0, trainable=False)

cell = tf.nn.rnn_cell.MultiRNNCell(
    [tf.nn.rnn_cell.LSTMCell(hidden_size[i], state_is_tuple=True) for i in range(num_layer)], 
    state_is_tuple=True)
initial_state = cell.zero_state(test_batch_size, tf.float32)

embeddings = tf.Variable(tf.random_uniform([len(char2id) + 1, embedding_size], -1.0, 1.0))
embedded = tf.nn.embedding_lookup(embeddings, X)

outputs, last_states = tf.nn.dynamic_rnn(cell, embedded, initial_state=initial_state)

outputs = tf.reshape(outputs, [-1, hidden_size[1]])
logits = tf.layers.dense(outputs, units=len(char2id) + 1)
probs = tf.nn.softmax(logits)
targets = tf.reshape(Y, [-1])

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))
params = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('./GA'))

def poetry_generate_with_topic(head):
    states_ = sess.run(initial_state)
    gen = ''
    c = '['
    i = 0
    while c != ']':
        gen += c
        x = np.zeros((test_batch_size, 1))
        x[:, 0] = char2id[c]
        probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
        probs_ = np.squeeze(probs_)
        pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
        while pos not in id2char:
            pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))

        c = id2char[pos]
        if i<len(head):
            c=head[i]
            i+=1

    return gen[1:]

print('Validation_loss = ', Validation_loss[-1])
print('Loss = ', Loss[-1])
print(poetry_generate_with_topic('边塞'))

We have 3391676 words
We have 8010 different words in dataset
We have 34 words
We have 32 different words in dataset


100%|████████████████████████████████████████████████████████████████████████████████| 203/203 [01:05<00:00,  3.06it/s]


INFO:tensorflow:Restoring parameters from ./GA\50_epoch
Validation_loss =  nan
Loss =  4.711365
边塞深携处主由，晚风一枕胜花残。时年一夜无人疎，亦喜当时一日看。


In [9]:
print(poetry_generate_with_topic('边塞'))

边塞晚江烟不醒，犯涛烟叶共闲风。回亭草里能闻柳，何忧今来解手篇。


In [10]:
print(poetry_generate_with_topic('边塞'))

边塞无人识道心，无殊和彻作平明。今朝莫费空身好，鲁答伤乘又偶慙。


In [12]:
print(poetry_generate_with_topic('长歌'))

长歌幸结瑞孱芽，醉尽灵田定太平。贤口已登轩久了，我渠自说下唐仙。


In [15]:
print(poetry_generate_with_topic('大漠'))

大漠山扃一日开，小舟飞泊碧瑶台。佳人何日飘为醉，卧待烟阴月上风。


In [16]:
print(poetry_generate_with_topic('临安'))

临安略远几时言，萤散峰头数隐路。诸子可怜年下相，三生千里西江去。


In [19]:
print(poetry_generate_with_topic('靖康'))

靖康门去好词初，月里巖台广铁肤。台下元来犹异入，自披扶笛进封朝。


In [20]:
print(poetry_generate_with_topic('楼兰'))

楼兰北静入无车，竹粉摇暄尚凭花。金烛忽盐清碧嶂，犹劳藕雨睡晨凉。


In [21]:
def poetry_generate():
    states_ = sess.run(initial_state)
    
    gen = ''
    c = '['
    while c != ']':
        gen += c
        x = np.zeros((test_batch_size, 1))
        x[:, 0] = char2id[c]
        probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
        probs_ = np.squeeze(probs_)
        pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
        while pos not in id2char:
            pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))

        c = id2char[pos]
    
    return gen[1:]
    #t.delete('1.0','end')
    #t.insert("insert", gen[1:])

In [26]:
print(poetry_generate())
print(poetry_generate())
print(poetry_generate())
print(poetry_generate())

洛川不见愁西架，淮雨乾髅上故船。寺里相逢鹤留大，松关水宇且钻衣。
禅家旧二茆芷群，最孝觚裳滋未传。想照清江和虺决，拜之随祸在冰中。
石舫俱人对白云，少年修壤病流频。我来坐引人劳说，万象情间箇事愁。
莫臾摇到翠云中，抵白波头拥玉池。再羨渠吹白青马，行人精海不能还。


In [28]:
print(poetry_generate())
print(poetry_generate())
print(poetry_generate())
print(poetry_generate())

雨散山光水芋礽，聚中时复洗人舟。旧人一曲幷前着，得尽佳心泛泪斜。
臙萸火瀑每依骚，散腹仙和曳宇中。骑梓行人宜奉比，为君安买一株涂。
菊花射我江霄水，吾故有他何处圆。商郎晚中诗语否，也听射文诗夫歌。
独送扁舟五海东，昭关太守百年炎。汉家承学羣恩制，细取深斋寄垒阳。
